In [1]:
from astropy.table import Table, join

import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

## Stellar mass catalogs from SDSS ancillary data
I re-examined the various methods that were used to compute stellar masses for a large sample of SDSS galaxies. I settled on two methods that I thought stood out from the rest: Granada (Conroy, Gunn & White) and Wisconsin (Chen, et al.). The former fits ugriz imaging with flexible stellar populations in a variety of configurations (along with a redshift given by the spectra). The latter uses a PCA analysis of a wide portion of individual spectra (3500 - 6000A? I forgot). 

The problem with the latter is that I can't find any documentation to determine whether the quoted stellar masses are for the fiber or if they've been scaled to the galaxy. Scaling to the galaxy would come with a significant error considering the assumptions that have to go in and the questionable photometry needed to convert the M/L ratio M.  

The problem with the former is that, under the assumption these are fairly robust measurements of the galaxy stellar mass, they will be for the entire galaxy and not individual clumps, whereas the Wisconsin method (if they don't try to scale the mass measurment from individual spectra up to the whole galaxy) could reflect the clump mass. 

I've downloaded both catalogs (where, for the Granada I've chosen "Wide No Dust" -- will explain later). 

In [2]:
wisc = Table.read('../data/final_clumpy_spectra_WiscBC03_highzgal.csv')
gran = Table.read('../data/final_clumpy_spectra_GranWideNoDust_highzgal.csv')
print gran.colnames
print 
print wisc.colnames

['OBJID', 'DR12objid', 'RA', 'DEC', 'ra2', 'dec2', 'specObjID', 'plate', 'mjd', 'fiberID', 'z', 'zErr', 'clump_dist', 'norm_clump_dist', 'logMass', 'logMass_median', 'logMass_err', 'metallicity_mean', 'metallicity_err', 'dust1_mean', 'dust1_err', 'tau_mean', 'tau_err', 'age', 'age_mean', 'ssfr', 'ssfr_mean']

['OBJID', 'DR12objid', 'RA', 'DEC', 'ra2', 'dec2', 'specObjID', 'plate', 'mjd', 'fiberID', 'z', 'zErr', 'clump_dist', 'norm_clump_dist', 'mstellar_median', 'mstellar_err', 'vdisp_median', 'vdisp_err', 'warning']


In [3]:
# My original list had 171 spectra so both catalogs are missing 5
print len(wisc), len(gran)

167 167


In [4]:
# There are 14 galaxies that are not well fit in Wisconsin Method
np.sum(wisc['warning']>0)

14

In [5]:
# There are 11 galaxies that are not well fit in the Granada Method
np.sum(gran['logMass_median']<=0)

11

In [6]:
# Of the galaxies that are not well-fit, only 3 overlap between the two samples.
granIDs = set(gran[gran['logMass_median']<=0]['OBJID'])
wiscIDs = set(wisc[wisc['warning']>0]['OBJID'])

granIDs & wiscIDs

{587731186200018973, 587731186725486612, 8647474690339438846}

## This is my FAVORITE galaxy! 
There are actually 4 spectra but both catalogs have only 1 spectrum and they both get it totally wrong. This galaxy has to be done individually, I guess. 

In [7]:
wisc[wisc['OBJID']==587731186200018973] #
gran[gran['OBJID']==587731186200018973] #

OBJID,DR12objid,RA,DEC,ra2,dec2,specObjID,plate,mjd,fiberID,z,zErr,clump_dist,norm_clump_dist,logMass,logMass_median,logMass_err,metallicity_mean,metallicity_err,dust1_mean,dust1_err,tau_mean,tau_err,age,age_mean,ssfr,ssfr_mean
int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
587731186200018973,1237657190901284914,351.0962,-0.1084575,351.0889,-0.10817,1232914866113636352,1095,52521,198,0.008917743,3.166824e-06,4.985298,1.522758,7.51054155905,-9999.0,-9999.0,-9999.0,-9999.0,0.0,0,-9999.0,-9999.0,1.29934740067,-9999.0,-0.285143464804,-9999.0


## This gal is has two spectra -- Both methods fail on the bright clump
1108-397-53227

In [8]:
gran[gran['OBJID']==587731186725486612] #

OBJID,DR12objid,RA,DEC,ra2,dec2,specObjID,plate,mjd,fiberID,z,zErr,clump_dist,norm_clump_dist,logMass,logMass_median,logMass_err,metallicity_mean,metallicity_err,dust1_mean,dust1_err,tau_mean,tau_err,age,age_mean,ssfr,ssfr_mean
int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
587731186725486612,1237663543680631053,324.9864,0.3232627,324.9854,0.322708,1247606277450786816,1108,53227,397,0.04983082,5.680749e-06,4.171423,1.02965,8.45076905666,-9999.0,-9999.0,-9999.0,-9999.0,0.0,0,-9999.0,-9999.0,1.29934740067,-9999.0,-0.285143464804,-9999.0
587731186725486612,1237663543680631055,324.9864,0.3232627,324.9865,0.3232599,1114759421507954688,990,52465,431,0.04987694,5.928548e-06,0.6812232,0.1681492,9.6804029105,9.6804029105,0.0359611749856,0.00777835373733,0.0003786907975,0.150150150061,0,1.13522033475,0.191394042035,1.69919931889,1.79805034152,-0.806965410709,-0.81810904812


In [9]:
wisc[wisc['OBJID']==587731186725486612] #

OBJID,DR12objid,RA,DEC,ra2,dec2,specObjID,plate,mjd,fiberID,z,zErr,clump_dist,norm_clump_dist,mstellar_median,mstellar_err,vdisp_median,vdisp_err,warning
int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64
587731186725486612,1237663543680631055,324.9864,0.3232627,324.9865,0.3232599,1114759421507954688,990,52465,431,0.04987694,5.928548e-06,0.6812232,0.1681492,9.83458688881,0.128614496876,117.534575462,34.4426238537,0
587731186725486612,1237663543680631053,324.9864,0.3232627,324.9854,0.322708,1247606277450786816,1108,53227,397,0.04983082,5.680749e-06,4.171423,1.02965,0.0,0.0,0.0,0.0,8


## This is a super low SB galaxy that is probably incorrectly broken into several SDSS objects. 

Granada method fails on the spectra that seems to be on a clump and a red star. (691-52199-123)
Wisconsin method fails on the spectra covering the faintest blob [20 mag] (1495-52944-55)

In [10]:
gran[gran['OBJID']==8647474690339438846] #

OBJID,DR12objid,RA,DEC,ra2,dec2,specObjID,plate,mjd,fiberID,z,zErr,clump_dist,norm_clump_dist,logMass,logMass_median,logMass_err,metallicity_mean,metallicity_err,dust1_mean,dust1_err,tau_mean,tau_err,age,age_mean,ssfr,ssfr_mean
int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
8647474690339438846,1237663782591791113,11.38851,-1.105878,11.39023,-1.10353,1221616009916475392,1085,52531,53,0.01366896,5.746615e-06,0.0,0.0,8.99305513243,8.98049869232,0.0218453047546,0.0279484681528,0.00340692586985,0.150150150061,0,8.44084237883,1.84808354841,2.41249394417,2.31289159944,-0.571290135384,-0.562833759937
8647474690339438846,1237663782591791110,11.38851,-1.105878,11.39126,-1.101733,778030682503931904,691,52199,123,0.01377012,3.565432e-06,0.0,0.0,8.64954186217,-9999.0,-9999.0,-9999.0,-9999.0,0.0,0,-9999.0,-9999.0,13.1198301315,-9999.0,-33.0,-9999.0
8647474690339438846,1237663782591791116,11.38851,-1.105878,11.39584,-1.104012,1683235528406755328,1495,52944,55,0.01372564,7.528681e-06,0.0,0.0,6.71340495267,6.71774374353,0.0660243678424,0.0139399862315,0.00457239118957,0.150150150061,0,9.32967690255,1.01302276345,1.29934740067,1.3034148604,-0.285143464804,-0.289745761008


In [11]:
wisc[wisc['OBJID']==8647474690339438846] #

OBJID,DR12objid,RA,DEC,ra2,dec2,specObjID,plate,mjd,fiberID,z,zErr,clump_dist,norm_clump_dist,mstellar_median,mstellar_err,vdisp_median,vdisp_err,warning
int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64
8647474690339438846,1237663782591791110,11.38851,-1.105878,11.39126,-1.101733,778030682503931904,691,52199,123,0.01377012,3.565432e-06,0.0,0.0,8.24816737026,0.193798275121,263.918018341,97.7764129639,0
8647474690339438846,1237663782591791113,11.38851,-1.105878,11.39023,-1.10353,1221616009916475392,1085,52531,53,0.01366896,5.746615e-06,0.0,0.0,8.48975027778,0.161218654887,352.610244751,44.7744655609,0
8647474690339438846,1237663782591791116,11.38851,-1.105878,11.39584,-1.104012,1683235528406755328,1495,52944,55,0.01372564,7.528681e-06,0.0,0.0,0.0,0.0,0.0,0.0,8


In [12]:
gran_bad = gran[gran['logMass_median']<=0]
wisc_bad = wisc[wisc['warning']>0]


wisc_good_gran_bad = join(gran_bad, wisc, keys=['OBJID','DR12objid','RA',
                                                'DEC','ra2','dec2','specObjID',
                                                'plate','mjd','fiberID','z',
                                                'zErr','clump_dist',
                                                'norm_clump_dist'])
gran_good_wisc_bad = join(wisc_bad, gran, keys=['OBJID','DR12objid','RA',
                                                'DEC','ra2','dec2','specObjID',
                                                'plate','mjd','fiberID','z',
                                                'zErr','clump_dist',
                                                'norm_clump_dist'])

## Let's compare stellar masses for the ones they both fit well

In [16]:
combo = join(wisc, gran, keys=['OBJID','DR12objid','RA','DEC',
                               'ra2','dec2','specObjID',
                               'plate','mjd','fiberID','z','zErr',
                               'clump_dist','norm_clump_dist'])

# Save the combined mass catalogs for later 
combo.write('../data/final_clumpy_spectra_2masscatalogs.csv', format='ascii.csv')

In [ ]:
good_masses = combo[(combo['warning']==0)&(combo['logMass_median']>0)]

In [ ]:
plt.scatter(good_masses['logMass_median'], good_masses['mstellar_median'])
plt.plot((8,11), (8,11), ls='--', lw=3, color='red')
plt.xlabel('Granada', fontsize=20)
plt.ylabel('Wisconsin', fontsize=20)

In [ ]:
high_mass = (good_masses['logMass_median'] > 10) & (good_masses['mstellar_median'] > 10)

print np.sum(high_mass)

In [ ]:
good_masses[high_mass]

## Test the GANDALF code on a galaxy that has a more robust mass measurement

I ran GANDALF on the spectrum of the galaxy center, `spec-0383-51818-0294.fit`, using the Milky Way value of E(B-V) that Vihang gave me from the Schlegel maps (0.0397).
I want to see if the output makes any sense and if I can sort of reconstruct the mass of this galaxy. 

In [ ]:
test_galaxy = good_masses[np.where(good_masses['OBJID']==588015508191445089)[0][0]]
test_galaxy